In [1]:
import sys
import platform
import telnetlib
import time
import re
import os
import datetime
import uao_decode
#import psycopg2
import pyte
import commands
import traceback

ImportError: No module named 'uao_decode'

In [ ]:

stream = pyte.Stream()
screen = pyte.Screen(80, 24)
stream.attach(screen)



def ip2ipnum(ip):
  if ip=="":
    return 0
  ip_num = 0
  cnt = 3
  l = map(int,ip.split("."))
  #錯誤檢測，IPv4 不可能小於４
  if len(l)<4:
    return 0
  for b in l:
    ip_num = ip_num + b*pow(256,cnt)
    cnt  = cnt -1
  return ip_num

class PTTDB:
  database=""
  user=""
  password=""
  host=""
  port=""
  conn = None
  cur = None
  table = None
  def __init__(self,table):
    self.table=table
    self.id = 0
  #啟用DB
  def connect(self):
    self.database = 'ptt_user'
    self.user = os.environ.get('AHA_PG_USER')
    self.password = os.environ.get('AHA_PG_PWD')
    self.host = os.environ.get('AHA_PG_IP')
    self.port = os.environ.get('AHA_PG_PORT')

    self.conn = psycopg2.connect(database=self.database, user=self.user, password=self.password, host=self.host, port=self.port)
    self.cur = self.conn.cursor()

  def start_record(self,ptt_id,total_cnt,time):
    sql = "INSERT INTO  user_crawler (id,total_cnt,status,start_dt) VALUES ('%s',%d,%d,'%s');"%(ptt_id,total_cnt,1,time)
    sql = sql+"SELECT currval(pg_get_serial_sequence('user_crawler','job_no'));"
    self.cur.execute(sql)
    self.conn.commit()
    self.id = int(self.cur.fetchone()[0])

  def end_record(self):
    sql = "UPDATE user_crawler SET end_dt=NOW(), status=0 WHERE job_no=%d"%(self.id)
    self.cur.execute(sql)
    self.conn.commit()

  def add_record(self,current_user):
    #sql = "UPDATE user_crawler SET record_cnt = record_cnt+1, current_time=NOW(), current_user='%s' WHERE job_no=%d"%(current_user,self.id )
    sql = "UPDATE user_crawler SET record_cnt = record_cnt+1 WHERE job_no=%d"%(self.id )
    self.cur.execute(sql)
    self.conn.commit()

  def error(self):
    sql = "UPDATE user_crawler SET end_dt=NOW(), status=2 WHERE job_no=%d"%(self.id)
    self.cur.execute(sql)
    self.conn.commit()



  def insertUSER(self,record,startTime):
    data = {}
    data['id'] = "'"+record[0]+"'"
    data['status'] = "'"+record[1]+"'"
    data['ip'] = "'"+record[2]+"'"
    data['login_cnt'] = record[4]
    data['article_cnt'] = record[5]
    data['bad_article_cnt'] = record[6]
    data['money'] = record[7]
    data['money_type'] = "'"+record[8]+"'"
    data['trace_time'] = "'"+startTime+"'"
    data['ip_num'] = str(ip2ipnum(record[2]))
    #print record[3]
    data['login_time'] = "'"+datetime.datetime.strptime(record[3],"%m/%d/%Y %H:%M:%S").strftime("%Y-%m-%d %H:%M:%S")+"'"

    columns = ",".join(list(data.keys()))
    values = ",".join(list(data.values()))
    sql = "INSERT INTO ptt_user (%s) VALUES (%s)"%(columns,values)
    #print sql
    self.cur.execute(sql)
    self.conn.commit()

  #結束DB
  def close(self):
    self.cur.close()
    self.conn.close()


'''
connet to PTT.cc

# https://dev.haiku-os.org/ticket/8482
#define PAGE_UP_KEY 0x21
#define PAGE_DOWN_KEY 0x36
#define LEFT_ARROW_KEY_CODE "\033OD"
#define RIGHT_ARROW_KEY_CODE "\033OC"
#define UP_ARROW_KEY_CODE "\033OA"
#define DOWN_ARROW_KEY_CODE "\033OB"
#define LEFT_ARROW_KEY_CODE "\033[D"
#define RIGHT_ARROW_KEY_CODE "\033[C"
#define UP_ARROW_KEY_CODE "\033[A"
#define DOWN_ARROW_KEY_CODE "\033[B"

#define CTRL_LEFT_ARROW_KEY_CODE "\033O5D"
#define CTRL_RIGHT_ARROW_KEY_CODE "\033O5C"

'''

class TELNET:
  duration = 0.5
  def key_left(self):
    self.telnet.write('\033[D')
    return self
  def key_right(self):
    self.telnet.write('\033[C')
    return self
  def key_down(self):
    self.telnet.write('\033[B')
    return self
  def key_up(self):
    self.telnet.write('\033[A')
    return self
  def key_control(self,key):
    self.telnet.write(chr(ord(key) - ord('a') + 1))
    return self
  def page_down(self):
    self.telnet.write('0x36')
    return self

  def __init__(self,db):
    self.pttDB = db

  def waitForPTT(self,s):
    self.screen = ""
    temp = ""
    while s not in temp:
      time.sleep(self.duration)
      self.screen = self.screen + self.telnet.read_very_eager().decode('uao_decode')
      temp = self.filterControlCode(self.screen)
    #print self.screen


  def waitForPTT2(self,s):
    self.screen = ""
    temp = ""
    while s not in temp:
      time.sleep(self.duration)
      self.screen = self.screen + self.telnet.read_very_eager().decode('uao_decode')
      temp = self.filterControlCode(self.screen)
      if u"[呼叫器]" in temp:
        return False
    #print self.screen
    return True



  def login(self,host, user, passwd):
    self.telnet.open(host)
    self.screen =''
    self.waitForPTT(u'請輸入代號')
    self.telnet.write('%s\r' % user)
    self.waitForPTT(u'請輸入您的密碼')
    self.telnet.write('%s\r' % passwd)
    self.telnet.write('\r')
    self.telnet.write('N\r')



  def gotoMainList(self):
    self.screen = ""
    while u'休閒聊天區' not in self.screen:
      time.sleep(self.duration)
      self.screen = self.screen + self.telnet.read_very_eager().decode('uao_decode')
      self.key_left()
    #print self.screen


  def gotoUserList(self,f):
    #開始循環讀取   根據list.log內的資料
      current_user=""

      try:
        self.telnet = telnetlib.Telnet()
        self.login('ptt.cc', self.user,'tteesstt')
        #確定是在主畫面
        self.gotoMainList()
        #選擇聊天區塊
        self.telnet.write("\T")
        self.key_right()
        self.telnet.write('q')
        self.key_right()

        #開始處理每一個User

        for line in f.xreadlines():
          self.pt = self.pt+1
          now_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
          current_user =line.strip()
          self.readUser(current_user)
          self.pttDB.add_record(current_user)
          a ="%s - [%s][%s] %s %d/%d,percent:%0.2f%%"%(self.user,self.start_time,now_time,current_user,self.pt,self.total_ppl,(self.pt*100.0)/self.total_ppl)
          print a
          with open(self.path+self.user+".log","w+") as f2:
            f2.write(a+"\n")
        self.disconnect()
        return (True,f)
      except EOFError: #re-run
        print 'telnet connection closed'
        self.telnet = None
        time.sleep(10)
        return (False,f)
      except:
        print 'Strange'
        self.telnet = None
        with open(self.path+'user_error.log','a+') as f2:
          now_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
          f2.write('\n ----- %s crawler:%s ptt id:%s-----\n'%(now_time,self.user,current_user))
          f2.write(traceback.format_exc())
          print traceback.print_exc()
          f2.write('\n ----- \n')
        #with open(self.path+"strang.log","a+") as f3:
        #  f3.write("%s %s\n"%(self.user,current_user))
        time.sleep(60)
        return (False,f)



  def readUser(self,user):
    self.waitForPTT(u'請輸入使用者代號:')
    self.telnet.write(user+'\r')
    if self.waitForPTT2(u'請按任意鍵繼續')==False: #意外發生
      #self.waitForPTT(u'[呼叫器]')
      self.key_right()
      return
    data = self.screen
    with open(self.path+'current_user','w+') as f:
      f.write(data)
    self.pttDB.insertUSER(self.parse(data),self.start_time)
    self.telnet.write('q')
    self.waitForPTT(u'[呼叫器]')
    self.key_right()


  def filterControlCode(self,data):
    D = re.sub('\x1b[[0-9;]*[mABCDHJKsu]','',data)
    D = re.sub('[\r]','',D)
    return D

 #取出重要資訊
  def parse(self,data):
    #為了修正有人簽名黨亂七八糟，用象棋戰機作為分隔線
    D = self.filterControlCode(data).split(u"《象棋戰績》")[0]
    #print D
    #有可能找不到IP  e.g., police012
    ip_pattern = re.compile(u'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}')
    if len(ip_pattern.findall(D))>0:
      ip = ip_pattern.findall(D)[-1]
    else:
      ip = ""
    time_pattern = re.compile(u'\d{1,2}\/\d{1,2}\/\d{4} \d{1,2}:\d{1,2}:\d{2}')
    if time_pattern.search(D) is not None:
      time = time_pattern.search(D).group(0)
    else:
      time = "1/1/1970 0:0:0"
    login_cnt_pattern = re.compile(u'《登入次數》(\d+) 次')
    login_cnt = login_cnt_pattern.search(D).group(1)
    article_cnt_pattern = re.compile(u'《有效文章》(\d+) 篇 \(退:(\d+)\)')
    m = article_cnt_pattern.search(D)
    article_cnt = m.group(1)
    bad_article_cnt = m.group(2)
    id_pattern = re.compile(u'《ＩＤ暱稱》(\w+) \(([^\n]*)\) *《')
    m = id_pattern.search(D)
    id = m.group(1)
    status = m.group(2).replace("'","''")
    money_pattern = re.compile(u'《經濟狀況》([^\(\)]+) \(\$(\d+)\)')
    m = money_pattern.search(D)
    if m is not None:
      money_type = m.group(1)
      money = m.group(2)
    else:
      money_pattern = re.compile(u'《經濟狀況》([\u4e00-\u9fa5_a-zA-Z0-9]+)')
      m = money_pattern.search(D)
      money_type = m.group(1)
      money = str(-1)
    return [id,status,ip,time,login_cnt,article_cnt,bad_article_cnt,money,money_type]

  def disconnect(self):
    self.telnet.write('\r\r')
    self.key_left().key_left().key_left()
    self.telnet.write('G\r')
    self.telnet.write('y\r\r')
    self.telnet.close()

  def init(self):
    self.path = "/home/aha/Project/PTT/log/USER/"
    self.pt = 0
    status,result = commands.getstatusoutput('wc -l %s'%self.userlist_path)
    self.total_ppl = int(result.split(" ")[0])
    self.start_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    os.system('rm /home/aha/Project/PTT/log/current_user')
    os.system('clear')
    self.pttDB.connect()
    self.pttDB.start_record(self.user,self.total_ppl,self.start_time)


  def main(self,path,user,log_path):
    self.userlist_path = path
    self.user = user
    self.path = log_path
    self.init()
    endFlag = False
    with open(self.userlist_path,'r') as f2:
      while endFlag==False:
        endFlag,f2 =  self.gotoUserList(f2)
    self.pttDB.end_record()

  def record_error_status(self):
    self.pttDB.error()


if __name__ == '__main__':
  user = sys.argv[2]
  path = sys.argv[1]
  telnet = TELNET(PTTDB('ptt_user'))
  try:
    telnet.main(path,user,'/home/aha/Project/PTT/log/')
  except:
    telnet.record_error_status()
    traceback.print_exc()



